In [1]:
import os,nbloader,warnings,sys,time
warnings.filterwarnings("ignore")

import numpy as np
import pickle

from multigoal_infoGAIL import train_info_imitation_learning

import itertools
from multiprocessing import Process

if __name__ == "__main__":
    print ("Python version is [%s]"%(sys.version_info[0]))

Python version is [2]


In [2]:
class grid_maker(object):
    def __init__(self,*_arg):
        self.arg = _arg
        self.nArg = len(self.arg) # Number of total lists
        _product = itertools.product(*self.arg); _nIter = 0
        for x in _product: _nIter += 1
        self.nIter = _nIter
        self.paramList = ['']*self.nIter
        self.idxList = ['']*self.nIter
        _product = itertools.product(*self.arg);
        for idx,x in enumerate(_product):
            self.paramList[idx] = x

In [3]:
def get_properIdx(_processID,_maxProcessID,_nTask):
    ret = []
    if _processID > _nTask: return ret
    if _processID > _maxProcessID: return ret
    m = (_nTask-_processID-1) // _maxProcessID
    for i in range(m+1):
        ret.append(i*_maxProcessID+_processID)
    return ret
if __name__ == "__main__":
    maxProcessID,nTask = 8,11
    print ("\nmaxProcessID:[%d], nTask:[%d]"%(maxProcessID,nTask ))
    for processID in range(maxProcessID):
        ids = get_properIdx(_processID=processID,_maxProcessID=maxProcessID,_nTask=nTask)
        print (" processID:[%d] %s"%(processID,ids))


maxProcessID:[8], nTask:[11]
 processID:[0] [0, 8]
 processID:[1] [1, 9]
 processID:[2] [2, 10]
 processID:[3] [3]
 processID:[4] [4]
 processID:[5] [5]
 processID:[6] [6]
 processID:[7] [7]


In [4]:
kl_targ_list=[3e-3]
entcoeff_list=[1e-1, 1e-3, 1e-5, 1e-7]
n_mixture_list=[4]
epi_size_list=[300]
seed_list=[1,2,3,4,5,6,7,8,9,10]
logpostcoeff_list=[1e-3]
    
def get_multigoal_config(_processID=0,_maxProcessID=2,_maxGPU=1,_DO_SHUFFLE=False):
    # Get total configurations
    _G = grid_maker(kl_targ_list,entcoeff_list,n_mixture_list,epi_size_list,logpostcoeff_list,seed_list)
    # Get current configurations
    _ids = get_properIdx(_processID,_maxProcessID,_nTask=_G.nIter)
    _paramsList = list(_G.paramList[i] for i in _ids) 
    # Set GPU ID
    _GPU_ID = (_processID % _maxGPU)
    # Suffle if required
    if _DO_SHUFFLE:
        shuffle(_paramsList)
    return _paramsList,_GPU_ID

if __name__ == "__main__":
    processID = 3
    maxProcessID = 10
    maxGPU = 1
    paramsList,GPU_ID = get_multigoal_config(processID,maxProcessID,maxGPU)
    print ("GPU_ID:[%d]"%(GPU_ID))
    for pIdx,params in enumerate(paramsList): # For all current configurations
        print (pIdx,params) 

GPU_ID:[0]
(0, (0.003, 0.1, 4, 300, 0.001, 4))
(1, (0.003, 0.001, 4, 300, 0.001, 4))
(2, (0.003, 1e-05, 4, 300, 0.001, 4))
(3, (0.003, 1e-07, 4, 300, 0.001, 4))


In [5]:
def train_imitation_learning_infoGAIL_multiprocessing(parameter_list=None,GPU_ID=0,verbose=False):
    for pIdx,params in enumerate(parameter_list): # For all current configurations
        kl_targ,entcoeff,n_mixture,demo_size,logpostcoeff,seed=params
        train_info_imitation_learning(seed,entcoeff,logpostcoeff,n_mixture,demo_size,verbose=verbose)

if __name__ == "__main__":
    processID = 4
    maxProcessID = 5
    maxGPU = 4
    parameter_list,GPU_ID = get_multigoal_config(processID,maxProcessID,maxGPU)
#     train_imitation_learning_infoGAIL_multiprocessing(parameter_list=parameter_list,GPU_ID=GPU_ID,verbose=True)

In [6]:
class worker_class(Process):
    def __init__(self,_idx=0,_maxProcessID=8,_maxGPU=8,_name='worker',_FLAG='',_period=1,_maxTick=5,_VERBOSE=True):
        Process.__init__(self)
        # Initialize class
        self.idx = _idx
        self.maxProcessID = _maxProcessID
        self.maxGPU = _maxGPU
        self.name = _name
        self.FLAG = _FLAG # Running flag 
        self.VERBOSE = _VERBOSE
        # Initialize Process 
        self.setName = 'T_'+self.name 
        self.args = (self.FLAG,)
        self.daemon = True # Make sure that each child is killed when the parent is dead. (?!)
        # Print
        if self.VERBOSE:
            print ("[%s] Instantiated."%(self.name))
    def run(self):
        print ("Starting [%s]"%(self.name))
        time.sleep(1e-1)
        # Get configurations
        parameter_list,GPU_ID = get_multigoal_config(self.idx,self.maxProcessID,self.maxGPU)
        print ("processID:[%d/%d] GPU_ID:[%d] #Config:[%d]"
               %(self.idx,self.maxProcessID,GPU_ID,len(parameter_list)))
        #Run
        train_imitation_learning_infoGAIL_multiprocessing(parameter_list=parameter_list,GPU_ID=GPU_ID,verbose=True)
        print ("[%s] Done."%(self.name))
        
if __name__ == "__main__":
    print ("worker_class defined.")

worker_class defined.
